# Import libs und Daten

In [55]:
import pandas as pd
from datetime import datetime
import numpy as np
# XML-Lib laden
import xml.etree.ElementTree as ET
from datawrapper import Datawrapper
dw = Datawrapper(access_token = "79263c731ee5ea069033d6c0e7f62798e440a05b21a43a4b1f1ae765a7eab433")
from datetime import datetime
from datetime import date
import random
import os
import glob
os.getcwd()
os.chdir(r"C:\\Users\\c_knoop\\Desktop\\BTW_2021")

In [56]:
os.getcwd()
os.chdir(r"C:\\Users\\c_knoop\\Desktop\\BTW_2021")

In [57]:
#url_xml = "./analyse_alt/rohdaten/wahl_gemeinden.xml"
#url_xml = "./test_daten/F.210906.124700.GE.xml"
url_xml = "./test_daten/F.210906.132800.WK.xml"
root = ET.parse(url_xml).getroot()
ergebnisse_2017_wk = pd.read_excel("./data/BW17_Ergebnisse_ORIGINALE.xlsx", sheet_name="BW17_Ergebnisse_WK")
wahlergebnisse_2021 = pd.read_csv("./data/BTW_2017_GE.csv")
direktbewerber = pd.read_excel("./data/statistik-sachsen_bundestagswahl2021_direktbewerber.xlsx", header=1)

In [58]:
datenstand = "24.09.2021, 14:26 Uhr."

# Herauslesen Daten Dataframe long

In [59]:
#HELPER
# Ordnet die Beschreibung des Tags <Gruppenergebnis> einer Klarbzeichung zu
# diese Beschreibung ist unter dem Attribut Position abgespeichert

# Check ob sich in einem String eine Nummer befindet
def has_numbers(inputString):
    return any(char.isdigit() for char in inputString)


In [60]:
# Dict Parteien 

In [61]:
# Sicherheitskopie, falls veränderte Partei-Namen für Probleme sorgen
# Offizielle Listennummer mit Parteiname BTW 2021
# parteien_dict = {"1":"AfD", "2":"CDU", "3":"DIE LINKE", "4":"SPD", "5":"FDP", "6":"GRÜNE", \
#                 # kleinere Parteien (sonstige)
#                 "7":"Tierschutzpartei", "8":"Die PARTEI", "9":"NPD", "10":"FREIE WÄHLER", \
#                  "11":"PIRATEN", "12":"ÖDP", "13":"V-Partei³", "14":"MLPD", "15":"dieBasis", 
#                  "16": "Bündnis C", "17": "III. Weg", "18":"DKP", "19":"Die Humanisten", \
#                  "20":"Gesundheitsforschung", "21":"Team Todenhöfer", "22":"Volt", \
#                  # kleinere Parteien ohne Landesliste
#                  "23": "ASMUSfddB", "24":"AufrechterDemokrat", "25":"Brähmig", "26":"BüSo", \
#                  "27":"Daniel Richter", "28":"Direktkandidat Daniel Zimmet", "29":"Direktkandidat Frank Hannig", \
#                  "30": "DK Schöllner", "31":"Ein Leipziger für Leipzig", "32":"#KLIMAfirst", \
#                  "33": "Internationalistisches Bündnis", "34": "Internationalistisches Bündnis", \
#                  "35": "Internationalistisches Bündnis", "36":"Ischgl", "37":"KARLKOMMT", \
#                  "38": "LKR", "39": "Marcus Fuchs - Querdenken 351", "40":"Queck", "41":"REICHEL",\
#                  "42":"Röhder", "43":"UNO - Menschrecht - FRIEDEN für Alle", "44":"WIR sind LEIPZIGER", \
#                  "45":"www.zeitkaufhaus.de Martin Bayer", \
#                  # Bezeichner für gültige und ungültige Stimmen
#                 "GUE": "gültige Stimmen", "UNG": "Ungültige Stimmen"

#                 }

In [62]:
# # Offizielle Listennummer mit Parteiname BTW 2021
parteien_dict = {"1":"AfD", "2":"CDU", "3":"Die Linke", "4":"SPD", "5":"FDP", "6":"Grüne", \
                # kleinere Parteien (sonstige)
                "7":"Tierschutzpartei", "8":"Die Partei", "9":"NPD", "10":"Freie Wähler", \
                 "11":"PIRATEN", "12":"ÖDP", "13":"V-Partei³", "14":"MLPD", "15":"dieBasis", 
                 "16": "Bündnis C", "17": "III. Weg", "18":"DKP", "19":"Die Humanisten", \
               "20":"Gesundheitsforschung", "21":"Team Todenhöfer", "22":"Volt", \
                 # kleinere Parteien ohne Landesliste
                 "23": "ASMUSfddB", "24":"AufrechterDemokrat", "25":"Brähmig", "26":"BüSo", \
                 "27":"Daniel Richter", "28":"Direktkandidat Daniel Zimmet", "29":"Direktkandidat Frank Hannig", \
                 "30": "DK Schöllner", "31":"Ein Leipziger für Leipzig", "32":"#KLIMAfirst", \
                 "33": "Internat. Bündnis", "34": "Internat. Bündnis", \
                 "35": "Internat. Bündnis", "36":"Ischgl", "37":"KARLKOMMT", \
                 "38": "LKR", "39": "Marcus Fuchs - Querdenken 351", "40":"Queck", "41":"Reichel",\
                 "42":"Röhder", "43":"UNO - Menschrecht - Frieden für Alle", "44":"Wir sind Leipziger", \
                 "45":"www.zeitkaufhaus.de Martin Bayer", \
                 # Bezeichner für gültige und ungültige Stimmen
                "GUE": "gültige Stimmen", "UNG": "Ungültige Stimmen"

                }

In [63]:


# jedes elem in root ist der Tag <Gebietsergebnis>, in dem immer eine komplette Gemeinde steckt. 
def read_xml(root):
    
    # Variablen in Datensatz
    erststimmen = "DIREKT"
    zweitstimmen = "LISTE"


    
    # zwei Dataframes werden erstellt, eins für die Ergebnisse der Parteien, 
    # ein anderes für Wahlstatistiken etc. 
    
    # DF Ergebnisse
    cols_result = ["KreisId", "KreisName", "Partei", "ParteiId", "Stimmart", "Stimmen", "Anteil", "Stand"]
    df_results = pd.DataFrame(columns=cols_result)
    
    # DF Statistiken
    col_stats = ["KreisId", "KreisName","Statistik", "Stimmen", "Anteil", "Stand"]
    df_stats = pd.DataFrame(columns=col_stats)
    
    # DF Wahlkreis-Sieger
    col_winner = ["KreisId", "KreisName", "Sieger"]
    df_winner = pd.DataFrame(columns=col_winner)
    
    # Durchlauf durch alle <Gruppenergebnis>, sammeln der Gemeindedaten
    for elem in root:
        
        kreis_id = elem.attrib.get("KreisNr")
        stand = elem.attrib.get("Stand") 
        kreis_nm = elem.attrib.get("KreisName")
        
        # Unterpunkt ist <Gruppenergebnis>, hier findet sich unter "Position" die Beschreibung des jew. Datensatzes
        for sub in elem: 
            position = sub.attrib.get("Position")
            
            # Check ob position eine Nummer ist - dann handelt es sich nämlich um ein Partei-Stimmergebnis
            # dann wird das result-Dataframe befüllt
            if (has_numbers(position)) | (position == "UNG") | (position == "GUE"): 
                partei = parteien_dict[position]
                
                for node in sub: 
                    kind_of_votes = node.attrib.get("Stimmart")
                    if kind_of_votes == erststimmen: 
                        stimmart = "Erststimme"
                    else: 
                        stimmart = "Zweitstimme"
                    count_votes = node.attrib.get("Stimmen")
                    votes_share = node.attrib.get("Anteil")
                    ser_results = pd.Series([kreis_id, kreis_nm, partei, position, stimmart, count_votes, votes_share, stand], index=cols_result)
                    df_results = df_results.append(ser_results, ignore_index=True)
            
            # wenn Buchstaben-Kennung wird das statistische Dataframe befüllt
            
            elif position=="SIEGER": 
                info_node = sub[0]
                sieger = info_node.attrib.get("Sieger")
                ser_win = pd.Series([kreis_id,kreis_nm, sieger], index=col_winner)
                df_winner = df_winner.append(ser_win, ignore_index=True)
            
            else:
                
                info_node = sub[0]
                stimmen = info_node.attrib.get("Stimmen")
                anteil = info_node.attrib.get("Anteil")
                
                if position == "WB": 
                    statistik = "Wahlberechtigte"
                    anteil = "100"
                elif position == "WAE":
                    statistik = "Wähler"
              
                else:
                    print(position)
                    print("Fehler")
               
                ser_stats = pd.Series([kreis_id, kreis_nm, statistik, stimmen, anteil, stand], index=col_stats)
                df_stats = df_stats.append(ser_stats, ignore_index=True)
        
    return df_results, df_stats, df_winner
                    
                    
            #if position
            
           

            
            
            
df_ergebnisse, df_daten, df_sieger = read_xml(root)

## richtige Datentypen vergeben

In [64]:

# richtige Datentypen vergeben
df_ergebnisse["Stimmen"] = df_ergebnisse["Stimmen"].astype(int)
df_ergebnisse["Anteil"] = df_ergebnisse["Anteil"].astype(float)

df_daten["Stimmen"] = df_daten["Stimmen"].astype(int)
df_daten["Anteil"] = df_daten["Anteil"].astype(float)

# Tabelle Erststimmen pro Wahlkreis ausspielen

## Direktbewerber filtern, Name und Vorname drehen

In [65]:
direktbewerber = pd.read_excel("./data/statistik-sachsen_bundestagswahl2021_direktbewerber.xlsx", header=1)
# Direktbewerber säubern
direktbewerber.rename(columns={"Wahlkreis 151 Nordsachsen":"KreisId", "Nummer":"ParteiId", \
                              "Wahlvorschlag \nKurzbezeichnung":"Partei"}, inplace=True)
df_dir = direktbewerber[["KreisId", "Partei", "ParteiId",  "Name"]]

In [66]:
df_dir.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205 entries, 0 to 204
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   KreisId   205 non-null    object
 1   Partei    205 non-null    object
 2   ParteiId  205 non-null    object
 3   Name      205 non-null    object
dtypes: object(4)
memory usage: 6.5+ KB


In [67]:
# Parteinamen nach SZ-Wunsch geändert
def change_party_name(el): 
    try: 
        return parteien_dict[str(el)]
    except: 
        el
df_dir["Partei"] = df_dir["ParteiId"].map(change_party_name)

<ipython-input-67-28afb1718607>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dir["Partei"] = df_dir["ParteiId"].map(change_party_name)


In [68]:

def new_name(el):
    
    try:
        new_list = el.split(", ")
    
        new_name = new_list[1] + " " + new_list[0]
       
        return new_name
    except: 
        return el


df_dir["Name"] = df_dir["Name"].map(new_name)


<ipython-input-68-c4e47e9a63e1>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dir["Name"] = df_dir["Name"].map(new_name)


In [69]:
df_dir["Name_Partei"] = df_dir["Name"] + ' (' + df_dir["Partei"] + ')'
#df_dir["Name_Partei2"] = df_dir["Name"] +'\n' + df_dir["Partei"]

# Id in string wandeln
df_dir["KreisId"] = df_dir["KreisId"].astype(str)
df_dir["ParteiId"] = df_dir["ParteiId"].astype(str)

In [70]:
# Erststimmen herausfiltern
df_es = df_ergebnisse[df_ergebnisse["Stimmart"]=="Erststimme"]
partei_to_del = ['Ungültige Stimmen', 'gültige Stimmen']
df_es = df_es[~df_es["Partei"].isin(partei_to_del)]

## Kandidaten und Stimmenanteil verbinden

In [71]:
df_all_erststimmen = df_es.merge(df_dir, on=["KreisId", "ParteiId"])

In [72]:
# auf notwendige Columns eindampfen 
df_all_erststimmen = df_all_erststimmen[["KreisId", "Name_Partei", "Anteil"]]

## Test-Datensatz bauen

In [78]:
df_kandis = df_dir[["KreisId" ,"ParteiId", "Name_Partei"]]
df_kandis = df_kandis[df_kandis["ParteiId"].str.isnumeric()]

In [79]:
# Func for creating random numbers 
def create_random_numbers(el): 
    el_num = int(el)
    if el_num <=6: 
        random_num = random.uniform(8, 32)
    elif el_num <=10: 
        random_num = random.uniform(0, 7)
    else: 
        random_num = random.uniform(0, 2)
    return round(random_num,1)

In [80]:
df_kandis["Anteil"] = df_kandis["ParteiId"].map(create_random_numbers)

In [81]:
del df_kandis["ParteiId"]

In [82]:
df_kandis.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 190 entries, 0 to 204
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   KreisId      190 non-null    object 
 1   Name_Partei  190 non-null    object 
 2   Anteil       190 non-null    float64
dtypes: float64(1), object(2)
memory usage: 5.9+ KB


In [23]:
all_kreisid = df_kandis.KreisId.unique()
for ids in all_kreisid: 
    df_ = df_kandis[df_kandis["KreisId"]==ids]
    url = f"./data/testdateien_erststimmen/{ids}_erstimmen.csv"
    df_.to_csv(url, index=False)

## TODO: aktueller Datensatz wie Testdatensatz bauen1!!

## Wahlbeteiligung berechnen

In [24]:
def wahlbeteiligung(kreisid): 
    df_ = df_daten[df_daten["KreisId"]==kreisid]
    wahlb = round(df_.loc[df_["Statistik"]=="Wähler", "Stimmen"].item() / \
        df_.loc[df_["Statistik"]=="Wahlberechtigte", "Stimmen"].item() *100,1) 
    return wahlb

## Datawrapper-Grafiken erneuern

In [26]:
# Auflistung der Kreis_Ids und der dazugehörige Datawrapper-Code
dict_kreisId_dwId = {"155":"bwpEN", "156": "V4cTH", "157":"7SnKy", "158":"dwDUx", "161":"hRaJz"}

In [27]:
all_kreisid = ["155", "156", "157", "158", "161"]
for ids in all_kreisid: 
    df_ = df_kandis[df_kandis["KreisId"]==ids]
    # abfrage wird aus altem df_ergebnisse gezogen
    abfrage = df_ergebnisse[df_ergebnisse["KreisId"]==ids]["Stand"].iloc[0]
    # wenn keine Daten vorhanden, bleibt der Dummy stehen
    if df_.shape[0] == 0: 
        print(f"Keine Daten für WK {ids} vorhanden!")
    else: 
        chart_id = dict_kreisId_dwId[ids]
        dw.add_data(chart_id, df_)
        
        # UZ Wahlbeteiligung einbauen
        # Wahlbeteiligung und Quellen eintragen: 
        source_name = "<b style='color: grey'>Landeswahlleiter</b>"
        source_url = "https://wahlen.sachsen.de/bundestagswahl-2021.html"
        byline = "Christoph Knoop | Die Mehrwertmacher GmbH"
        wbt = wahlbeteiligung(ids)
        unterzeile = f"<b>Wahlbeteiligung: {str(wbt).replace('.', ',')} Prozent (+3,5).</b>"
        dw.update_description(chart_id,source_name=source_name, source_url=source_url,\
                              byline=byline, intro=unterzeile)
        
        #string_date1 = f"Datenstand: {datetime.now().strftime('%d.%m.%Y, %H:%M')}."
        string_date1 = f"<b>Datenstand: {datenstand}</b>"
        string_date2 = f"<b> {abfrage} ausgezählt.</b>"
        string_final = string_date1 + string_date2
        props = {'annotate': {'notes': string_final}}
        dw.update_metadata(chart_id, props)
        dw.publish_chart(chart_id, display=False)
#endregion
print("Aktualisierung Erststimmen Wahlkreise abgeschlossen.")
        
        

Chart updated!
Chart's metadata updated!
Chart updated!
Chart's metadata updated!
Chart updated!
Chart's metadata updated!
Chart updated!
Chart's metadata updated!
Chart updated!
Chart's metadata updated!
Aktualisierung Erststimmen Wahlkreise abgeschlossen.


## Datawrapper als PNG ausspielen 

In [30]:
all_kreisid = ["155", "156", "157", "158", "161"]
for ids in all_kreisid: 
    chart_id = dict_kreisId_dwId[ids]
    folder = "./data/png_wahlkreise/"
    file = f"{ids}_erststimme.png"
    path = folder + file
    dw.export_chart(chart_id, width=768, zoom=3, border_width=30, filepath = path)
    
print("Erststimmen Wahkreise als png ausgespielt.")    

File exported at data\png_wahlkreise\155_erststimme.png
File exported at data\png_wahlkreise\156_erststimme.png
File exported at data\png_wahlkreise\157_erststimme.png
File exported at data\png_wahlkreise\158_erststimme.png
File exported at data\png_wahlkreise\161_erststimme.png
Erststimmen Wahkreise als png ausgespielt.


## PNGs auf Server übertragen

In [31]:
from ftplib import FTP 
import pysftp 
pw = "Fair67?sea" 
name = "christophknoop" 
hostname = "app.dmwm.de" 
hostname2 = "christophknoop@app.dmwm.de" 
hostname3 = "https://datasets.app.dmwm.de" 
# ftp = FTP(hostname) 
# ftp.login(user=name, passwd = pw) 
cnopts = pysftp.CnOpts() 
cnopts.hostkeys = None 

# Ort der hochzuladenen Dateien

#folder = "./data/png_wahlkreise"
folder = r"C:\\Users\\c_knoop\\Desktop\\BTW_2021\\data\\png_wahlkreise\\*.png"
#remote_folder = r"\\var\\www\\christophknoop\\wahlkarte\\png\\wahlkreise\\"
remote_folder = r"/var/www/christophknoop/wahlkarte/png/wahlkreise/"
with pysftp.Connection(hostname, username=name, password=pw, cnopts=cnopts) as sftp: 
    sftp.cwd('/var/www/christophknoop/wahlkarte/png/wahlkreise') 
    for filepath in glob.iglob(folder):
        sftp.put(filepath)  
               # upload file to public/ on remote     #sftp.put("./Daten/Abschlussdateien/nicht_kumuliert.csv")
sftp.close() 
print("PNGs Erststimmen Wahlkreise an Server gesendet.")

PNGs Erststimmen Wahlkreise an Server gesendet.


# Tabelle Zweitstimme pro Wahlkreis ausspielen

## Zweiststimmen filtern, df erstellen

In [83]:
# richtiges Arbeitsverzeichnis wieder herstellen
os.getcwd()
os.chdir(r"C:\\Users\\c_knoop\\Desktop\\BTW_2021")

In [84]:
df_zs = df_ergebnisse[df_ergebnisse["Stimmart"]=="Zweitstimme"]
df_zs = df_zs[["KreisId", "Partei", "Stimmen", "Anteil"]]

In [85]:
list_all_KreisIds = ["155", "156", "157", "158", "161"]
df_zs = df_zs[df_zs["KreisId"].isin(list_all_KreisIds)]

## "sonstige" berechnen

In [86]:
def reduce_number_of_parties(df_zs): 
    liste_standing_terms = ["Ungültige Stimmen", "gültige Stimmen", "CDU", "Die Linke", \
                            "AfD", "FDP", "Grüne", "SPD"]
       
    df_ = df_zs[~df_zs["Partei"].isin(liste_standing_terms)]
    df_rest = df_zs[df_zs["Partei"].isin(liste_standing_terms)]
    df_flat = df_.groupby(["KreisId"], as_index=False).agg({"Stimmen":"sum", "Anteil":"sum"})
    df_flat["Partei"] = "sonstige"
    df_complete = df_flat.append(df_rest)
    
    df_complete = df_complete.sort_values(by="KreisId", ascending=True)
    return df_complete
        
     
    
    
    
df_zs_flat = reduce_number_of_parties(df_zs)    

## korrekter Anteil "sonstige" berechnen

In [87]:
for ids in list_all_KreisIds: 
    df_zs_flat.loc[(df_zs_flat["KreisId"]==ids) & \
                   (df_zs_flat["Partei"]=="sonstige"), "Anteil"] = \
    round(df_zs_flat.loc[(df_zs_flat["KreisId"]==ids) & \
                  (df_zs_flat["Partei"]=="sonstige"), "Stimmen"].item()/ \
        df_zs_flat.loc[(df_zs_flat["KreisId"]==ids) & \
                      (df_zs_flat["Partei"]=="gültige Stimmen"), "Stimmen"].item() * 100,1)

## unnötige Zeilen und columns entfernen

In [88]:
rows_to_delete = ["Ungültige Stimmen", "gültige Stimmen"]
df_zs_flat = df_zs_flat[~df_zs_flat["Partei"].isin(rows_to_delete)]

## historische Daten einarbeiten

### historische Daten Cleaning

In [89]:
dfh = ergebnisse_2017_wk.copy()
# richtige Formate vergeben
dfh["WK-Nr"] = dfh["WK-Nr"].astype(str)
dfh = dfh.replace("x", "0")
dfh["AfD_1"] = dfh["AfD_1"].astype(int)
dfh["NPD_1"] = dfh["NPD_1"].astype(int)

dfh["PIRATEN_1"] = dfh["PIRATEN_1"].astype(int)
dfh["FREIE WÄHLER_1"] = dfh["FREIE WÄHLER_1"].astype(int)
dfh["BüSo_1"] = dfh["BüSo_1"].astype(int)
dfh["MLPD_1"] = dfh["MLPD_1"].astype(int)
dfh["ÖDP_1"] = dfh["ÖDP_1"].astype(int)
dfh["Die PARTEI_1"] = dfh["Die PARTEI_1"].astype(int)
dfh["FREIE WÄHLER_1"] = dfh["FREIE WÄHLER_1"].astype(int)
dfh["Bürgerkandidat_1"] = dfh["Bürgerkandidat_1"].astype(int)
dfh["Der Auserwählte_1"] = dfh["Der Auserwählte_1"].astype(int)
dfh["Heimat Region Zwickau_1"] = dfh["Heimat Region Zwickau_1"].astype(int)
dfh["Held_1"] = dfh["Held_1"].astype(int)
dfh["Hoyer_1"] = dfh["Hoyer_1"].astype(int)
dfh["Internationalistisch. Bündnis_1"] = dfh["Internationalistisch. Bündnis_1"].astype(int)
dfh["RINK_1"] = dfh["RINK_1"].astype(int)
dfh["SGP_1"] = dfh["SGP_1"].astype(int)
dfh["Wir sind LEIPZIGER_1"] = dfh["Wir sind LEIPZIGER_1"].astype(int)

# richtige Spaltennamen zum Mergen vergeben

### Nur Zweitstimmen und gültige Zweitstimmen filtern

In [90]:
df_hist = dfh.iloc[:, [1,29,30, 31,32,33,34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47]]
df_long = df_hist.melt(id_vars=["WK-Nr"], var_name = "Partei", value_name="Stimmen")
list_all_KreisIds = ["155", "156", "157", "158", "161"]
df_long = df_long[df_long["WK-Nr"].isin(list_all_KreisIds)]

In [91]:
df_long["Partei"] = df_long["Partei"].str[:-2]

In [92]:
df_long["Partei"] = df_long["Partei"].str.replace("ungültige", "Ungültige Stimmen").replace("gültige", "gültige Stimmen")

df_long.rename(columns={"WK-Nr":"KreisId"}, inplace=True)

### "Sonstige" berechnen

In [94]:
df_long.Partei.unique()

array(['Ungültige Stimmen', 'gültige Stimmen', 'CDU', 'DIE LINKE', 'SPD',
       'AfD', 'GRÜNE', 'NPD', 'FDP', 'PIRATEN', 'FREIE WÄHLER', 'BüSo',
       'MLPD', 'BGE', 'DiB', 'ÖDP', 'Die PARTEI', 'Tierschutzpartei',
       'V-Partei³'], dtype=object)

In [95]:
all_parties = ["CDU", "AfD", "FDP", "SPD", "DIE LINKE", "GRÜNE"]

def reduce_number_of_parties_zs(df_zs): 
    liste_standing_terms = ["Ungültige Stimmen", "gültige Stimmen", "CDU", "DIE LINKE", \
                            "AfD", "FDP", "GRÜNE", "SPD"]
       
    df_ = df_zs[~df_zs["Partei"].isin(liste_standing_terms)]
    df_rest = df_zs[df_zs["Partei"].isin(liste_standing_terms)]
    #df_flat = df_.groupby(["KreisId"], as_index=False).sum()
    df_flat_ = df_.groupby(["KreisId"], as_index=False).agg({"Stimmen":"sum"})
    df_flat_["Partei"] = "sonstige"
    df_complete = df_flat_.append(df_rest)
    
    df_complete = df_complete.sort_values(by="KreisId", ascending=True)
    return df_complete
        
     
    
    
    
df_long_all = reduce_number_of_parties_zs(df_long)    

### Anteile in historischen Daten berechnen

In [96]:

df_long_all["gültige_stimmen"] = 0
for ids in list_all_KreisIds: 
    gültige_stimmen = df_long_all.loc[(df_long_all["KreisId"]==ids) & (df_long_all["Partei"]=="gültige Stimmen"), "Stimmen"].item()
    df_long_all.loc[df_long_all["KreisId"]==ids, "gültige_stimmen"] = gültige_stimmen

In [97]:
df_long_all["Anteil_2017"] = round(df_long_all["Stimmen"] / df_long_all["gültige_stimmen"] * 100,1)

## aktuelle und historische Daten zusammenführen, +/- Zeichen setzen, Partei und Veränderung in einen string packen

In [98]:
all_parties_2017 = ["CDU", "FDP", "AfD", "DIE LINKE", "SPD", "GRÜNE", "sonstige"]
all_parties_2021 = ["CDU", "FDP", "AfD", "Die Linke", "SPD", "Grüne", "sonstige"]
df_long_all = df_long_all[df_long_all["Partei"].isin(all_parties_2017)]
df_zs_flat = df_zs_flat[df_zs_flat["Partei"].isin(all_parties_2021)]

In [99]:
# Schreibweise 2017er-Datensatz an aktuellen Datensatz anpassen
df_long_all["Partei"] = df_long_all["Partei"].str.replace("DIE LINKE", "Die Linke").replace("GRÜNE", "Grüne")

In [100]:
df_all = df_zs_flat.merge(df_long_all[["KreisId", "Partei", "Anteil_2017"]], on=["KreisId","Partei"])

In [101]:
# Veränderugen berechnen, mit +/- versehen und mit Parteinamen verbinden
df_all["Veränderung"] = round(df_all["Anteil"] - df_all["Anteil_2017"],1)
df_all["Veränderung_string"] = df_all["Veränderung"].apply(lambda x: "+"+str(x) if x >0 else str(x))

In [102]:

df_all["Parteiname"] = df_all["Partei"] + ' (' + df_all["Veränderung_string"]+')'

In [103]:
df_final = df_all[["KreisId", "Parteiname", "Anteil"]]


## Tabelle sortieren

Ist hier per Hand nötig, da "sonstige" immer die letzte Column sein sollte. 



In [104]:
def create_rank_sorter(df): 
    df["sorter"] = df["Anteil"].rank(ascending=False)
    df.loc[df["Parteiname"].str.contains("sonstige"), "sorter"] = 100
    df_ = df.sort_values(by="sorter")
    
    return df_
    


## Grafiken anlegen

In [105]:
zs_dict = {"155": "uJaIJ", "156":"M1VEB", "157":"ynwmP", "158":"Pk2Zx", "161": "h8f5r" }

In [108]:
all_kreisid = ["155", "156", "157", "158", "161"]
for ids in all_kreisid: 
    df_ = df_final[df_final["KreisId"]==ids]
    # abfrage wird aus altem df_ergebnisse gezogen
    abfrage = df_ergebnisse[df_ergebnisse["KreisId"]==ids]["Stand"].iloc[0]
    # wenn keine Daten vorhanden, bleibt der Dummy stehen
    if df_.shape[0] == 0: 
        print(f"Keine Daten für WK {ids} vorhanden!")
    else: 
        # df sortieren
        df_sorted = create_rank_sorter(df_)
        del df_sorted["sorter"]
        chart_id = zs_dict[ids]
        dw.add_data(chart_id, df_sorted)
        
        # Wahlbeteiligung und Quellen eintragen: 
        source_name = "<b style='color: grey'>Landeswahlleiter</b>"
        source_url = "https://wahlen.sachsen.de/bundestagswahl-2021.html"
        byline = "Christoph Knoop | Die Mehrwertmacher GmbH"
        wbt = wahlbeteiligung(ids)
        
        unterzeile1 = f"<b>Wahlbeteiligung: {str(wbt).replace('.', ',')} Prozent (+3,5).</b><br>"
        unterzeile2 = f"<b>Zahlen in Klammern: Vergleich zur Wahl 2017.</b>"
        unterzeile = unterzeile1 + unterzeile2
        dw.update_description(chart_id,source_name=source_name, source_url=source_url,\
                              byline=byline, intro=unterzeile)
        
        # Datenstand und Auszählungsstand eintragen
        string_date1 = f"<b>Datenstand: {datenstand}</b>"
        #string_date1 = datenstand
        string_date2 = f" <b>{abfrage} ausgezählt.</b>"
        string_final = string_date1 + string_date2
        props = {'annotate': {'notes': string_final}}
        dw.update_metadata(chart_id, props)
        dw.publish_chart(chart_id, display=False)
#endregion
print("Aktualisierung Zweitstimme Wahlkreise abgeschlossen.")

<ipython-input-104-cf2afc58753b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["sorter"] = df["Anteil"].rank(ascending=False)
c:\users\c_knoop\appdata\local\programs\python\python39\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


Chart updated!
Chart's metadata updated!
Chart updated!
Chart's metadata updated!
Chart updated!
Chart's metadata updated!
Chart updated!
Chart's metadata updated!
Chart updated!
Chart's metadata updated!
Aktualisierung Zweitstimme Wahlkreise abgeschlossen.


## PNGs ausspielen

In [109]:
all_kreisid = ["155", "156", "157", "158", "161"]
for ids in all_kreisid: 
    chart_id = zs_dict[ids]
    folder = "./data/png_wahlkreise_zweitstimme/"
    file = f"{ids}_zweitstimme.png"
    path = folder + file
    dw.export_chart(chart_id, width=768, zoom=3, border_width=30, filepath = path)
    
print("Zweitstimmen Wahkreise als png ausgespielt.")    

File exported at data\png_wahlkreise_zweitstimme\155_zweitstimme.png
File exported at data\png_wahlkreise_zweitstimme\156_zweitstimme.png
File exported at data\png_wahlkreise_zweitstimme\157_zweitstimme.png
File exported at data\png_wahlkreise_zweitstimme\158_zweitstimme.png
File exported at data\png_wahlkreise_zweitstimme\161_zweitstimme.png
Zweitstimmen Wahkreise als png ausgespielt.


## PNGs an Server schicken

In [110]:
from ftplib import FTP 
import pysftp 
pw = "Fair67?sea" 
name = "christophknoop" 
hostname = "app.dmwm.de" 
hostname2 = "christophknoop@app.dmwm.de" 
hostname3 = "https://datasets.app.dmwm.de" 
# ftp = FTP(hostname) 
# ftp.login(user=name, passwd = pw) 
cnopts = pysftp.CnOpts() 
cnopts.hostkeys = None 

# Ort der hochzuladenen Dateien

#folder = "./data/png_wahlkreise"
folder = r"C:\\Users\\c_knoop\\Desktop\\BTW_2021\\data\\png_wahlkreise_zweitstimme\\*.png"
with pysftp.Connection(hostname, username=name, password=pw, cnopts=cnopts) as sftp: 
    sftp.cwd('/var/www/christophknoop/wahlkarte/png/wahlkreise_zweitstimme') 
    for filepath in glob.iglob(folder):
        sftp.put(filepath)  
               # upload file to public/ on remote     #sftp.put("./Daten/Abschlussdateien/nicht_kumuliert.csv")
sftp.close() 
print("PNGs Zweitstimme Wahlkreise an Server gesendet.")

PNGs Zweitstimme Wahlkreise an Server gesendet.


# Parteien nach Stärke sortieren

Grundsätzlich haben wir **sechs freie Parteien-Slots** und andere Parteien. Die sechs gesetzten Parteien möchten wir jetzt nach Stärke verteilen.

**Zunächst nutzen wir dabei nur die Zweitstimmen**, bei Erststimmen müssen wir nochmal schauen 


Dafür nutze ich die Daten **vor** dem long to wide. Die Long-Daten finden sich in diesen Dataframes: 

df_es_flat 

df_zs_flat

## Zweitstimmen nach Stärke

In [ ]:
def sort_parties(df): 
    
    cols = ["AGS", "Ort", "Gewinner",\
            "Partei_1_Stimmen", "Partei_1_Anteil", "Partei_1_Name",\
            "Partei_2_Stimmen", "Partei_2_Anteil", "Partei_2_Name",\
            "Partei_3_Stimmen", "Partei_3_Anteil", "Partei_3_Name",\
            "Partei_4_Stimmen", "Partei_4_Anteil", "Partei_4_Name",\
            "Partei_5_Stimmen", "Partei_5_Anteil", "Partei_5_Name",\
            "Partei_6_Stimmen", "Partei_6_Anteil", "Partei_6_Name",\
            "Partei_andere_Stimmen", "Partei_andere_Anteil", "Partei_andere_Name", \
           "ungültige Stimmen", "gültige Stimmen"]
            
           
      
    new_df = pd.DataFrame(columns=cols)
    
    # jeder Ort wird durchlaufen
    for ags in df["AGS"].unique():
        dict_var = {}
        df_ = df[df["AGS"]==ags]
        ort = df_.iloc[0].Ort
        
        
        # "andere Parteien" erfassen
        andere = df_[df_["Partei"]=="andere Parteien"]
        if andere.shape[0]>0:
            andere_stimmen = andere.Stimmen.item()
            andere_anteil = andere.Anteil.item()
            andere_name = "andere Parteien"
        else:
            andere_stimmen = 0
            andere_anteil = 0
            andere_name = "andere Parteien"
            
        
        # ungültige Stimmen erfassen
        ungültig = df_[df_["Partei"]=="Ungültige Stimmen"]["Stimmen"].item()
        gültig = df_[df_["Partei"]=="gültige Stimmen"]["Stimmen"].item()
        
        
        # andere Parteien herausfiltern (da sie sonst ev. einen Platz in der Parteienrangfolge einnehmen)
        df_ = df_[~df_.Partei.isin(["Ungültige Stimmen", "gültige Stimmen", "andere Parteien"])]
            
        # Parteien nach Stimmen sortieren, höchste zuerst
        df_sort = df_.sort_values(by="Stimmen", ascending=False)
        # Gewinner als Klarnamen herausfiltern
        gewinner = df_sort.iloc[0]["Partei"]
        
        # Durchlauf durch die AGS, für jede Gemeinde sechs Parteien in Rangfolge setzen
        for i in range(len(df_sort.index)):
            j = i+1
            
            j = i+1
            var_stimmen = f"partei_{j}_Stimmen"
            var_anteil = f"partei_{j}_Anteil"
            var_name = f"partei_{j}_Name"
            val_stimmen = df_sort.iloc[i]["Stimmen"]
            val_anteil = df_sort.iloc[i]["Anteil"]
            val_name = df_sort.iloc[i]["Partei"]
            
            dict_var[var_stimmen] = val_stimmen
            dict_var[var_anteil] = val_anteil
            dict_var[var_name] = val_name
            
        
        new_ser = pd.Series([ags, ort, gewinner, \
                             dict_var['partei_1_Stimmen'],dict_var['partei_1_Anteil'], dict_var['partei_1_Name'],\
                             dict_var['partei_2_Stimmen'],dict_var['partei_2_Anteil'], dict_var['partei_2_Name'],\
                             dict_var['partei_3_Stimmen'],dict_var['partei_3_Anteil'], dict_var['partei_3_Name'],\
                             dict_var['partei_4_Stimmen'],dict_var['partei_4_Anteil'], dict_var['partei_4_Name'],\
                             dict_var['partei_5_Stimmen'],dict_var['partei_5_Anteil'], dict_var['partei_5_Name'],\
                             dict_var['partei_6_Stimmen'],dict_var['partei_6_Anteil'], dict_var['partei_6_Name'],\
                             andere_stimmen, andere_anteil, andere_name, \
                             ungültig, gültig
                            ], index=cols)
        
        new_df = new_df.append(new_ser, ignore_index=True)
    
    # richtige Datentypen vergeben
    new_df[["Partei_andere_Stimmen", "ungültige Stimmen", "gültige Stimmen"]] = \
        new_df[["Partei_andere_Stimmen", "ungültige Stimmen", "gültige Stimmen"]].astype(int)
    
    new_df["Partei_andere_Anteil"] = round(new_df["Partei_andere_Stimmen"] / new_df["gültige Stimmen"] * 100,1)
    
    return new_df
            
            
        
        
        
    
    
df_zs_rank = sort_parties(df_zs_flat)

## Erststimme nach Stärke

In [ ]:
df_es_rank = sort_parties(df_es_flat)   

In [ ]:
# ACHTUNG, nur für Demozwecke
# df_es_rank["Gewinner"] = "Name, Vorname (" + df_es_rank['Gewinner'] + ")"
# df_es_rank["Partei_1_Name"] = "Name, Vorname (" + df_es_rank['Partei_1_Name'] + ")"
# df_es_rank["Partei_2_Name"] = "Name, Vorname (" + df_es_rank['Partei_2_Name'] + ")"
# df_es_rank["Partei_3_Name"] = "Name, Vorname (" + df_es_rank['Partei_3_Name'] + ")"
# df_es_rank["Partei_4_Name"] = "Name, Vorname (" + df_es_rank['Partei_4_Name'] + ")"
# df_es_rank["Partei_5_Name"] = "Name, Vorname (" + df_es_rank['Partei_5_Name'] + ")"
# df_es_rank["Partei_6_Name"] = "Name, Vorname (" + df_es_rank['Partei_6_Name'] + ")"

## Partei nach Stärke mit WK-Daten mergen

In [ ]:
# Schritt 1 - präfix vergeben
#Präfix _1 für Erststimme, _2 für Zweitstimme hinzufügen
df_es_rank = df_es_rank.add_prefix("1_")
df_zs_rank = df_zs_rank.add_prefix("2_")
df_es_rank.rename(columns={"1_AGS":"AGS", "1_Ort":"Ort"}, inplace=True)
df_zs_rank.rename(columns={"2_AGS":"AGS", "2_Ort":"Ort"}, inplace=True)

In [ ]:
# Schritt 2 - alle DAten zusammenfassen in eine Wide-Tabelle

df_merge = df_es_rank.merge(df_zs_rank, on=["AGS", "Ort"])
df_merge = df_merge.merge(df_daten_wide, on=["AGS", "Ort"])

In [ ]:
df_merge.to_clipboard(decimal=",", index=False)
df_merge.to_csv("datawrapper_dummy.csv", index=False)

## Wahlergebnisse 2017 einarbeiten

In [ ]:
df_merge["2_Partei_1_2017_Anteil"] = 0
df_merge["2_Partei_2_2017_Anteil"] = 0
df_merge["2_Partei_3_2017_Anteil"] = 0
df_merge["2_Partei_4_2017_Anteil"] = 0
df_merge["2_Partei_5_2017_Anteil"] = 0
df_merge["2_Partei_6_2017_Anteil"] = 0
df_merge["2_Partei_andere_2017_Anteil"] = 0
    

In [ ]:
# dfh = wahlergebnisse_2021.copy()

# def create_old_results(df, dfh): 
#     for i in df.index: 
        
#         dict_col_names = {"AfD":"2017_AfD_%", "CDU":"2017_CDU_%", "LINKE":"2017_LINKE_%", "SPD":"2017_SPD_%", \
#                          "FDP":"2017_FDP_%", "GRÜNE":"2017_GRÜNE_%", "andere Parteien":"2017_andere_%"} 
#         partei_1 = df.at[i, "2_Partei_1_Name"]
#         anteil_2017_1 = 
        
        
        
# create_old_results(df_merge)

In [ ]:
dfh

# Erstimmen-Daten mit Wahlkreis-Nr. und Kandidaten verbinden

In [ ]:
df_k = pd.read_excel("./data/statistik-sachsen_bundestagswahl2021_direktbewerber.xlsx", header=1)

In [ ]:
# nur Kandidaten der sechs größeren Parteien behalten
liste_parteien = ["AfD", "CDU", "DIE LINKE", "FDP", "GRÜNE", "SPD"]
df_kandidaten = df_k[df_k["Wahlvorschlag \nKurzbezeichnung"].isin(liste_parteien)] 
df_kandidaten["Full_Name"] = df_kandidaten['Name'] +" (" + df_kandidaten['Wahlvorschlag \nKurzbezeichnung']+")"

df_kandidaten.rename(columns={"Wahlkreis 151 Nordsachsen":"Wahlkreis", "Nummer":"Listennummer", \
                             "Wahlvorschlag \nKurzbezeichnung": "Partei", \
                              "Wahlvorschlag \nLangbezeichnung": "Partei_lang", \
                             "Name":"Name", "Full_Name": "Name_lang"}, inplace=True)
# richtige Formate vergeben
df_kandidaten["Wahlkreis"] = df_kandidaten["Wahlkreis"].astype(str)
df_kandidaten["Listennummer"] = df_kandidaten["Listennummer"].astype(str)

In [ ]:
df_kandidaten

In [ ]:
def find_candidates(df_results, df_candidates):
    all_ags = df_results.AGS.unique()
    
    # Helper um mit Partie-Namen die Listennummer herauszufinden
    def return_party_nr(p_name): 
        for nr, name in parteien_dict.items():
            if p_name == name:
                return nr
        
    
    for ags in all_ags: 
        df_ = df_results[df_results["AGS"]==ags]
        winner = df_["1_Partei_1_Name"].item()
        number_winner = return_party_nr(winner)
        wknr = df_["WKNr"].item()
        name_winner = ""
        try:
            name_winner = df_kandidaten[(df_kandidaten["Wahlkreis"]==wknr) & \
                                    (df_kandidaten["Listennummer"]==number_winner)]["Name_lang"].item()
        except: 
            print(wknr)
            print(ags)
            print(number_winner)
            print(winner)
        percent_share_winner = 
    
        df_results.loc[df_results["AGS"]==ags, "1_Gewinner"] = name_winner + " (" ")"
    

    
    
    return df_results




        
df_fin = find_candidates(df_merge, df_kandidaten)

## Leipzig und Dresden haben je zwei Wahlkreise... Winner verändern

Hier muss ich auf Werte zurückgreifen, die 

In [ ]:
wknr = "164"
number_winner = "2"
df_kandidaten[df_kandidaten["Listennummer"]==1]